In [47]:
import numpy as np
import fiftyone as fo
import fiftyone.utils.random as four
import fiftyone.zoo as foz
import cv2
import pyautogui
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import pandas as pd
from ultralytics import YOLO

In [2]:
# A dataset with `ground_truth` detections and no tags
dataset = (
    foz.load_zoo_dataset("quickstart")
    .select_fields("ground_truth")
    .set_field("tags", [])
).clone()

four.random_split(dataset, {"train": 0.7, "test": 0.2, "val": 0.1}, seed=4)

Dataset already downloaded
You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [3]:
splits = ["train","val"]
label_field = "ground_truth"
export_dir = "yolov5_dataset"
classes = unique_labels = dataset.distinct("ground_truth.detections.label")
for split in splits:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

 100% |█████████████████| 140/140 [1.4s elapsed, 0s remaining, 90.2 samples/s]          
Directory 'yolov5_dataset' already exists; export will be merged with existing files
 100% |███████████████████| 20/20 [224.7ms elapsed, 0s remaining, 89.0 samples/s]      


In [44]:
model = YOLO("best.pt")

In [45]:
label = 'yogi'
results = model.predict(f"{label}.jpg", conf=0.5)

Loading best.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

image 1/1 C:\Users\knigh\Documents\Mercyhurst\R\YOLOv1\yogi.jpg: 640x640 (no detections), 795.3ms
Speed: 9.4ms preprocess, 795.3ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


In [46]:
annotated_image = results[0].plot()

# 1. Get the original dimensions of the annotated image.
# We'll use the shape of the image array (height, width, channels)
original_height, original_width, _ = annotated_image.shape

# 2. Get the screen resolution
screen_width, screen_height = pyautogui.size()

# 3. Calculate the aspect ratio
aspect_ratio = original_width / original_height

# 4. Calculate new dimensions that maintain the aspect ratio
if (screen_width / screen_height) > aspect_ratio:
    # Screen is wider, so scale by height
    new_height = screen_height
    new_width = int(new_height * aspect_ratio)
else:
    # Screen is taller, so scale by width
    new_width = screen_width
    new_height = int(new_width / aspect_ratio)

# 5. Resize the image with the new dimensions
resized_image = cv2.resize(annotated_image, (round(new_width*.8), round(new_height*.8)))

# 6. Display the resized image
cv2.imshow("YOLO Detections", resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
cv2.imwrite(f"{label}_annotated.jpg", annotated_image)
print(f"\nAnnotated image saved as '{label}_annotated.jpg'")


Annotated image saved as 'babu_annotated.jpg'


SyntaxError: invalid decimal literal (1798366427.py, line 274)